# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2151627269764,
 'week52high': 147.75,
 'week52low': 84.07,
 'week52highSplitAdjustOnly': 147.86,
 'week52lowSplitAdjustOnly': 85,
 'week52change': 0.5445700905924998,
 'sharesOutstanding': 16691749045,
 'float': 0,
 'avg10Volume': 70856585,
 'avg30Volume': 80683770,
 'day200MovingAvg': 131.9,
 'day50MovingAvg': 130.98,
 'employees': 151691,
 'ttmEPS': 4.6,
 'ttmDividendRate': 0.8578821902986637,
 'dividendYield': 0.00672982491456221,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-27',
 'nextEarningsDate': '2021-07-15',
 'peRatio': 27.60744101208148,
 'beta': 1.5118363117373743,
 'maxChangePercent': 47.64154609591075,
 'year5ChangePercent': 4.575809765584341,
 'year2ChangePercent': 1.739267904075762,
 'year1ChangePercent': 0.535816476468248,
 'ytdChangePercent': -0.04846123342243291,
 'month6ChangePercent': 0.021145362207080975,
 'month3ChangePercent': 0.04019745352988088,
 'month1ChangePercent': -0.0334498309552664,
 'day30ChangePercent'

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.535816476468248

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
        ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,139.060,0.537456,N/A
1,AAL,25.100,0.305145,N/A
2,AAP,198.900,0.340264,N/A
3,AAPL,127.241,0.555517,N/A
4,ABBV,115.760,0.267953,N/A
...,...,...,...,...
500,YUM,121.630,0.251756,N/A
501,ZBH,158.607,0.164664,N/A
502,ZBRA,509.700,0.82399,N/A
503,ZION,59.290,0.55292,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index()
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,139.060,0.537456,N/A
1,AAL,25.100,0.305145,N/A
2,AAP,198.900,0.340264,N/A
3,AAPL,127.241,0.555517,N/A
4,ABBV,115.760,0.267953,N/A
5,ABC,118.330,0.219746,N/A
6,ABMD,299.900,0.171188,N/A
7,ABT,112.360,0.252449,N/A
8,ACN,286.720,0.376675,N/A
9,ADBE,525.600,0.29704,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")
    
    try:
        float(portfolio_size)
    except:
        print('This is not a number! \nPlease try again: ')
        portfolio_size = input("Enter the size of your portfolio: ")
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 1000000
1000000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe

C:\Users\aryal\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,139.060,0.537456,143
1,AAL,25.100,0.305145,796
2,AAP,198.900,0.340264,100
3,AAPL,127.241,0.555517,157
4,ABBV,115.760,0.267953,172
5,ABC,118.330,0.219746,169
6,ABMD,299.900,0.171188,66
7,ABT,112.360,0.252449,177
8,ACN,286.720,0.376675,69
9,ADBE,525.600,0.29704,38


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns =  hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
        index = hqm_columns),
        ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.060,N/A,0.554694,N/A,0.209078,N/A,0.191594,N/A,0.0289036,N/A,N/A
1,AAL,24.300,N/A,0.305832,N/A,0.424454,N/A,0.192358,N/A,0.107246,N/A,N/A
2,AAP,203.440,N/A,0.337933,N/A,0.264791,N/A,0.132658,N/A,-0.0561216,N/A,N/A
3,AAPL,128.580,N/A,0.538501,N/A,0.021136,N/A,0.0401554,N/A,-0.0336226,N/A,N/A
4,ABBV,115.112,N/A,0.268689,N/A,0.0939857,N/A,0.0727926,N/A,-0.0241601,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.390,N/A,0.253403,N/A,0.140829,N/A,0.145194,N/A,-0.0200044,N/A,N/A
501,ZBH,160.034,N/A,0.158164,N/A,0.0904632,N/A,-0.00229718,N/A,-0.0871423,N/A,N/A
502,ZBRA,511.700,N/A,0.8463,N/A,0.362697,N/A,0.0611888,N/A,0.0376722,N/A,N/A
503,ZION,59.590,N/A,0.560532,N/A,0.369527,N/A,0.0441578,N/A,-0.0192036,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
               'One-Year',
               'Six-Month',
               'Three-Month',
               'One-Month'
               ]
hqm_dataframe.replace([None], 0, inplace = True)
# rv_dataframe.replace([None], 0, inplace = True)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.060,N/A,0.554694,0.730693,0.209078,0.526733,0.191594,0.794059,0.028904,0.722772,N/A
1,AAL,24.300,N/A,0.305832,0.445545,0.424454,0.871287,0.192358,0.80198,0.107246,0.956436,N/A
2,AAP,203.440,N/A,0.337933,0.487129,0.264791,0.647525,0.132658,0.544554,-0.056122,0.132673,N/A
3,AAPL,128.580,N/A,0.538501,0.708911,0.021136,0.132673,0.040155,0.231683,-0.033623,0.225743,N/A
4,ABBV,115.112,N/A,0.268689,0.4,0.093986,0.277228,0.072793,0.324752,-0.024160,0.285149,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.390,N/A,0.253403,0.368317,0.140829,0.386139,0.145194,0.6,-0.020004,0.330693,N/A
501,ZBH,160.034,N/A,0.158164,0.188119,0.090463,0.267327,-0.002297,0.120792,-0.087142,0.0554455,N/A
502,ZBRA,511.700,N/A,0.846300,0.912871,0.362697,0.813861,0.061189,0.293069,0.037672,0.774257,N/A
503,ZION,59.590,N/A,0.560532,0.736634,0.369527,0.821782,0.044158,0.245545,-0.019204,0.334653,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.060,N/A,0.554694,0.730693,0.209078,0.526733,0.191594,0.794059,0.028904,0.722772,0.693564
1,AAL,24.300,N/A,0.305832,0.445545,0.424454,0.871287,0.192358,0.80198,0.107246,0.956436,0.768812
2,AAP,203.440,N/A,0.337933,0.487129,0.264791,0.647525,0.132658,0.544554,-0.056122,0.132673,0.45297
3,AAPL,128.580,N/A,0.538501,0.708911,0.021136,0.132673,0.040155,0.231683,-0.033623,0.225743,0.324752
4,ABBV,115.112,N/A,0.268689,0.4,0.093986,0.277228,0.072793,0.324752,-0.024160,0.285149,0.321782
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.390,N/A,0.253403,0.368317,0.140829,0.386139,0.145194,0.6,-0.020004,0.330693,0.421287
501,ZBH,160.034,N/A,0.158164,0.188119,0.090463,0.267327,-0.002297,0.120792,-0.087142,0.0554455,0.157921
502,ZBRA,511.700,N/A,0.846300,0.912871,0.362697,0.813861,0.061189,0.293069,0.037672,0.774257,0.698515
503,ZION,59.590,N/A,0.560532,0.736634,0.369527,0.821782,0.044158,0.245545,-0.019204,0.334653,0.534653


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DXC,40.850,N/A,1.247283,0.980198,0.674639,0.978218,0.491925,0.99604,0.175410,0.988119,0.985644
1,NUE,111.860,N/A,1.450136,0.986139,0.983485,0.99802,0.731767,1,0.090722,0.940594,0.981188
2,F,16.155,N/A,1.166610,0.970297,0.751024,0.988119,0.302061,0.952475,0.343900,0.99604,0.976733
3,DVN,31.121,N/A,1.264420,0.982178,0.970607,0.99604,0.222763,0.873267,0.184290,0.990099,0.960396
4,SYF,51.390,N/A,1.081330,0.960396,0.569039,0.946535,0.265355,0.926733,0.102859,0.952475,0.946535
5,IRM,48.220,N/A,0.684188,0.837624,0.628427,0.966337,0.311773,0.958416,0.112117,0.958416,0.930198
6,COF,168.800,N/A,1.121718,0.968317,0.800298,0.992079,0.288716,0.946535,0.040988,0.794059,0.925248
7,AMAT,139.380,N/A,1.345860,0.984158,0.577352,0.954455,0.236622,0.893069,0.047684,0.831683,0.915842
8,NVDA,712.320,N/A,1.006448,0.946535,0.307667,0.722772,0.416186,0.990099,0.197828,0.992079,0.912871
9,SLB,37.603,N/A,0.652205,0.817822,0.575030,0.952475,0.241911,0.90297,0.124966,0.970297,0.910891


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

C:\Users\aryal\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DXC,40.850,489,1.247283,0.980198,0.674639,0.978218,0.491925,0.99604,0.175410,0.988119,0.985644
1,NUE,111.860,178,1.450136,0.986139,0.983485,0.99802,0.731767,1,0.090722,0.940594,0.981188
2,F,16.155,1238,1.166610,0.970297,0.751024,0.988119,0.302061,0.952475,0.343900,0.99604,0.976733
3,DVN,31.121,642,1.264420,0.982178,0.970607,0.99604,0.222763,0.873267,0.184290,0.990099,0.960396
4,SYF,51.390,389,1.081330,0.960396,0.569039,0.946535,0.265355,0.926733,0.102859,0.952475,0.946535
5,IRM,48.220,414,0.684188,0.837624,0.628427,0.966337,0.311773,0.958416,0.112117,0.958416,0.930198
6,COF,168.800,118,1.121718,0.968317,0.800298,0.992079,0.288716,0.946535,0.040988,0.794059,0.925248
7,AMAT,139.380,143,1.345860,0.984158,0.577352,0.954455,0.236622,0.893069,0.047684,0.831683,0.915842
8,NVDA,712.320,28,1.006448,0.946535,0.307667,0.722772,0.416186,0.990099,0.197828,0.992079,0.912871
9,SLB,37.603,531,0.652205,0.817822,0.575030,0.952475,0.241911,0.90297,0.124966,0.970297,0.910891


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()